# Iterating through all patients in all years to find peep>5

In [50]:
import pandas as pd
from datetime import datetime
import os
import pickle
from datetime import timedelta
from datetime import datetime
from datetime import date
from datetime import time
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import os

import json

import sys
from scipy.io import loadmat

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
import matplotlib.ticker as plticker

import seaborn as sns


import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PhysioMC/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
# from preprocessing import *
from ECG_module import *
from dataIO import *
from stage1_PPG_analysis import *

from importlib import reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# min_threshold = 5

In [3]:
%pwd

'/labs/kamaleswaranlab/Sepsis3_062321/mchan_analysis/disparities_O2/repo/stage3_interrater_analysis'

In [4]:
outputdir = '../../data/stage3/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [5]:
inputdir = '/labs/kamaleswaranlab/Sepsis3_062321/Files_cp_sftp_6_30_21/Files_cp_6_30_21/'

# load csv files David prepared

## load the demographic data, only look at black or white subjects for now

In [6]:
# demographic_filedir = '../../data/Pat_BedTime_List_some_on_BMI_2_6_22_dem.csv'
demographic_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_dem.csv'
df_demographic = pd.read_csv(demographic_filedir, index_col=0)  

# mask out subjects who are not black and white
# mask = (df_demographic['Race String']!='0.0')

mask = (df_demographic['Race String']=='African American  or Black') | (df_demographic['Race String']=='Caucasian or White')

print('only {:.2f}% is black or white'.format(mask.mean()*100))
df_demographic = df_demographic[mask]

only 43.47% is black or white


In [7]:
df_demographic

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String
0,A118-0520880083,2018-03-12 18:41:23,2018-03-12 23:13:00,18004189.0,2018-03-12 18:41:39,03/14/2018 20:20:00,False,True,African American or Black
1,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,01/18/2018 21:15:32,True,False,Caucasian or White
3,A088-0516735120,2018-01-23 19:18:40,2018-01-23 19:43:15,17395882.0,2018-01-23 19:17:04,01/26/2018 20:42:23,True,False,Caucasian or White
5,A051-0495488308,2017-05-22 21:25:08,2017-06-02 15:53:25,17655916.0,2017-05-22 21:21:50,06/02/2017 12:51:00,True,False,Caucasian or White
8,A121-0520783579,2018-03-11 15:52:58,2018-03-11 15:53:14,91393403.0,2018-03-11 15:48:46,03/14/2018 18:44:32,True,False,Caucasian or White
...,...,...,...,...,...,...,...,...,...
6266,A134-0515603037,2018-01-10 16:50:37,2018-01-31 04:03:15,49764871.0,2018-01-31 01:58:06,01/31/2018 21:23:39,False,True,African American or Black
6268,A119-0515603035,2018-01-10 16:50:35,2018-02-06 19:00:52,92559329.0,2018-02-06 21:53:05,02/07/2018 23:51:17,False,True,African American or Black
6269,A113-0515603017,2018-01-10 16:50:16,2018-03-03 01:42:37,96901673.0,2017-12-12 15:59:54,03/02/2018 17:03:03,False,True,African American or Black
6270,A020-0509637604,2017-11-02 15:46:44,2017-12-05 20:08:36,98391127.0,2017-12-05 18:12:04,12/11/2017 18:43:14,False,True,African American or Black


## load the oxygen saturation data
### TODO: need to add FiO2 and PaO2 data

In [8]:
# FiO2 and PaO2? (PaO2 / FiO2 < 400 indicates respiratory problem)
oxygen_sat_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_spo2_sao2.csv'
df_oxygen_sat = pd.read_csv(oxygen_sat_filedir, index_col=0)  

In [9]:
mask = (df_oxygen_sat['Black']==1) | (df_oxygen_sat['White']==1)
df_oxygen_sat = df_oxygen_sat[mask]

In [10]:
df_oxygen_sat['times'] =  pd.to_datetime(df_oxygen_sat['times'], format="%Y-%m-%d %H:%M:%S")


In [11]:
df_oxygen_sat = df_oxygen_sat.rename(columns = {'pat_id':'PAT_ID', 'csn':'CSN'})
df_oxygen_sat

,index,spo2,sao2,times,Black,White,PAT_ID,CSN,csn_idx
0,4,97.0,96.5,2017-07-17 12:29:03,0.0,1.0,1842529.0,3885207198,45128.0
1,5,98.0,96.5,2017-07-17 13:29:03,0.0,1.0,1842529.0,3885207198,45128.0
2,6,95.0,96.5,2017-07-17 14:29:03,0.0,1.0,1842529.0,3885207198,45128.0
3,7,97.0,96.5,2017-07-17 15:29:03,0.0,1.0,1842529.0,3885207198,45128.0
4,8,97.0,96.5,2017-07-17 16:29:03,0.0,1.0,1842529.0,3885207198,45128.0
...,...,...,...,...,...,...,...,...,...
27,30,100.0,98.9,2018-03-29 08:00:47,1.0,0.0,99962669.0,58104308087,107482.0
28,31,100.0,98.9,2018-03-29 09:00:47,1.0,0.0,99962669.0,58104308087,107482.0
29,32,100.0,98.9,2018-03-29 10:00:47,1.0,0.0,99962669.0,58104308087,107482.0
30,33,98.0,98.9,2018-03-29 11:00:47,1.0,0.0,99962669.0,58104308087,107482.0


In [12]:
df_oxygen_sat.describe()

,index,spo2,sao2,Black,White,PAT_ID,CSN,csn_idx
count,200960.000000,200960.000000,200960.000000,200960.000000,200960.000000,2.009600e+05,2.009600e+05,200960.000000
mean,269.355608,97.045106,96.870626,0.408360,0.591640,6.553404e+07,3.652402e+10,48579.365013
std,361.627686,4.305423,3.633344,0.491532,0.491532,3.403954e+07,2.047750e+10,29484.596870
min,0.000000,0.000000,26.550000,0.000000,0.000000,1.842529e+06,6.669707e+07,73.000000
25%,45.000000,96.000000,96.200000,0.000000,0.000000,2.485274e+07,1.352412e+10,22993.000000
50%,136.000000,98.000000,97.900000,0.000000,1.000000,8.832823e+07,4.666620e+10,46759.000000
75%,350.000000,100.000000,98.800000,1.000000,1.000000,9.786810e+07,5.546849e+10,70885.000000
max,4426.000000,100.000000,100.750000,1.000000,1.000000,9.996267e+07,5.811763e+10,142522.000000


In [13]:
df_oxygen_sat['CSN'].unique().shape

(2290,)

# load SpO2 and PaO2 csv

In [14]:
oxygen_sat_spo2_pao2_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_spo2_pao2.csv'
df_spo2_pao2_sat = pd.read_csv(oxygen_sat_spo2_pao2_filedir, index_col=0)  

mask = (df_spo2_pao2_sat['Black']==1) | (df_spo2_pao2_sat['White']==1)
df_spo2_pao2_sat = df_spo2_pao2_sat[mask]
df_spo2_pao2_sat = df_spo2_pao2_sat.rename(columns = {'pat_id':'PAT_ID', 'csn':'CSN'})
df_spo2_pao2_sat

,index,spo2,pao2,times,Black,White,PAT_ID,CSN,csn_idx
0,4,97.0,80.0,2017-07-17 12:29:03,0.0,1.0,1842529.0,3885207198,45128.0
1,5,98.0,80.0,2017-07-17 13:29:03,0.0,1.0,1842529.0,3885207198,45128.0
2,6,95.0,80.0,2017-07-17 14:29:03,0.0,1.0,1842529.0,3885207198,45128.0
3,7,97.0,80.0,2017-07-17 15:29:03,0.0,1.0,1842529.0,3885207198,45128.0
4,8,97.0,80.0,2017-07-17 16:29:03,0.0,1.0,1842529.0,3885207198,45128.0
...,...,...,...,...,...,...,...,...,...
20,21,100.0,143.0,2018-03-29 04:50:55,0.0,1.0,99963024.0,58104498087,82995.0
21,22,98.0,143.0,2018-03-29 05:50:55,0.0,1.0,99963024.0,58104498087,82995.0
22,23,100.0,143.0,2018-03-29 06:50:55,0.0,1.0,99963024.0,58104498087,82995.0
23,24,100.0,143.0,2018-03-29 07:50:55,0.0,1.0,99963024.0,58104498087,82995.0


In [15]:
df_spo2_pao2_sat['times'] =  pd.to_datetime(df_spo2_pao2_sat['times'], format="%Y-%m-%d %H:%M:%S")


In [16]:
df_spo2_pao2_sat.describe()

,index,spo2,pao2,Black,White,PAT_ID,CSN,csn_idx
count,201589.000000,201589.000000,201589.000000,201589.000000,201589.000000,2.015890e+05,2.015890e+05,201589.000000
mean,269.169741,97.046023,126.777645,0.408876,0.591124,6.554002e+07,3.652067e+10,48594.554028
std,362.015092,4.309443,65.725386,0.491628,0.491628,3.403344e+07,2.047610e+10,29490.600367
min,0.000000,0.000000,16.000000,0.000000,0.000000,1.842529e+06,6.669707e+07,73.000000
25%,45.000000,96.000000,84.000000,0.000000,0.000000,2.496497e+07,1.350943e+10,22993.000000
50%,136.000000,98.000000,111.000000,0.000000,1.000000,8.832823e+07,4.666620e+10,46759.000000
75%,350.000000,100.000000,148.000000,1.000000,1.000000,9.786810e+07,5.546849e+10,70885.000000
max,4426.000000,100.000000,531.000000,1.000000,1.000000,9.996302e+07,5.811763e+10,142522.000000


In [17]:
df_spo2_pao2_sat['CSN'].unique().shape

(2297,)

## load the ind csv
### compute BED Time Diff (to show that we have much more bed data)
### show the median the time diffs

In [18]:
ind_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_ind.csv'
df_ind = pd.read_csv(ind_filedir, index_col=0)  

mask = (df_ind['Race String']=='African American  or Black') | (df_ind['Race String']=='Caucasian or White')
df_ind = df_ind[mask]

In [19]:
df_ind['DateStart'] = pd.to_datetime(df_ind['DateStart'], format="%Y-%m-%d %H:%M:%S")
df_ind['DateEnd'] = pd.to_datetime(df_ind['DateEnd'], format="%Y-%m-%d %H:%M:%S")

## TODO: change BED Time Difference

In [20]:
BED_LOCATION_START = pd.to_datetime(df_ind['BED_LOCATION_START'], format="%Y-%m-%d %H:%M:%S")
BED_LOCATION_END = pd.to_datetime(df_ind['BED_LOCATION_END'], format="%m/%d/%Y %H:%M:%S")

df_ind['BED_LOCATION_START'] = BED_LOCATION_START
df_ind['BED_LOCATION_END'] = BED_LOCATION_END


In [21]:
DateStart = pd.to_datetime(df_ind['DateStart'], format="%Y-%m-%d %H:%M:%S")
DateEnd = pd.to_datetime(df_ind['DateEnd'], format="%m/%d/%Y %H:%M:%S")

df_ind['DateStart'] = DateStart
df_ind['DateEnd'] = DateEnd

df_ind['BED Time Difference'] = (DateEnd-DateStart).dt.total_seconds()

In [22]:
print('EHR time diff: {:.2f}hr'.format(df_ind['Time Difference'].median()/60/60))
print('Waveform time diff: {:.2f}hr'.format(df_ind['BED Time Difference'].median()/60/60))

EHR time diff: -0.06hr
Waveform time diff: 42.28hr


In [23]:
df_ind

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String,Time Difference,Time Found,True Time Found,CSN Exist Flag,CSN,BED Time Difference
0,A118-0520880083,2018-03-12 18:41:23,2018-03-12 23:13:00,18004189.0,2018-03-12 18:41:39,2018-03-14 20:20:00,False,True,African American or Black,1235.0,2018-03-12 18:20:48,2018-03-12 11:20:48,True,3.967156e+10,1.629700e+04
1,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7.998000e+03
2,A065-0489270594,2017-03-11 22:16:34,2017-03-14 14:50:58,96266988.0,2017-03-11 22:15:37,2017-03-14 10:58:53,False,True,African American or Black,1450.0,2017-03-11 21:52:24,2017-03-11 16:52:24,True,5.290317e+10,2.324640e+05
3,A088-0516735120,2018-01-23 19:18:40,2018-01-23 19:43:15,17395882.0,2018-01-23 19:17:04,2018-01-26 20:42:23,True,False,Caucasian or White,-20.0,2018-01-23 19:19:00,2018-01-22 22:19:00,True,5.754876e+10,1.475000e+03
5,A051-0495488308,2017-05-22 21:25:08,2017-06-02 15:53:25,17655916.0,2017-05-22 21:21:50,2017-06-02 12:51:00,True,False,Caucasian or White,-652.0,2017-05-22 21:36:00,2017-05-22 18:36:00,True,3.005482e+10,9.304970e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,A050-0519142630,2018-02-20 16:03:50,2018-02-20 16:10:55,92165161.0,2017-10-27 21:00:00,2017-10-28 11:54:32,True,False,Caucasian or White,9436187.0,2017-11-03 11:54:03,2017-10-27 16:54:03,True,4.521433e+10,4.250000e+02
6279,A050-0519143068,2018-02-20 16:11:08,1969-12-31 23:59:58,92165161.0,2017-10-27 21:00:00,2017-10-28 11:54:32,True,False,Caucasian or White,9436625.0,2017-11-03 11:54:03,2017-10-27 16:54:03,True,4.521433e+10,-1.519143e+09
6280,A075-0519140345,2018-02-20 15:25:45,2018-02-20 15:32:34,50293730.0,2017-10-27 15:19:55,2017-10-27 21:08:22,True,False,Caucasian or White,8883352.0,2017-11-09 19:49:53,2017-10-24 10:49:53,True,1.426466e+10,4.090000e+02
6283,A047-0519141716,2018-02-20 15:48:36,2018-02-20 15:52:32,23329164.0,2017-10-25 10:08:13,2017-10-28 11:24:36,False,True,African American or Black,9913831.0,2017-10-28 22:58:05,2017-10-18 18:58:05,True,1.089383e+10,2.360000e+02


In [24]:
df_ind['CSN'].unique().shape

(3644,)

In [25]:
df_ind['Time Difference']

0          1235.0
1         -1531.0
2          1450.0
3           -20.0
5          -652.0
          ...    
6278    9436187.0
6279    9436625.0
6280    8883352.0
6283    9913831.0
6284   -3218438.0
Name: Time Difference, Length: 5696, dtype: float64

# merge df_ind with df_oxygen_sat so I can get deBedTime, DateStart, DateEnd, PAT_ID, White	Black, CSN, SaO2, SpO2, oxy sat. times in one column

## TODO: merge using the CSN instead 

In [26]:
df_merged = df_ind.merge(df_oxygen_sat[['spo2', 'sao2', 'times', 'PAT_ID', 'CSN']], on=['CSN', 'PAT_ID'], how='outer')
df_merged

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String,Time Difference,Time Found,True Time Found,CSN Exist Flag,CSN,BED Time Difference,spo2,sao2,times
0,A118-0520880083,2018-03-12 18:41:23,2018-03-12 23:13:00,18004189.0,2018-03-12 18:41:39,2018-03-14 20:20:00,False,True,African American or Black,1235.0,2018-03-12 18:20:48,2018-03-12 11:20:48,True,3.967156e+10,16297.0,NaN,NaN,NaT
1,A118-0520896395,2018-03-12 23:13:15,2018-03-15 03:19:11,18004189.0,2018-03-12 18:41:39,2018-03-14 20:20:00,False,True,African American or Black,-453.0,2018-03-12 23:20:48,2018-03-12 11:20:48,True,3.967156e+10,187556.0,NaN,NaN,NaT
2,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,94.500000,92.4,2018-01-10 17:16:00
3,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,96.000000,92.4,2018-01-10 18:16:00
4,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,95.333333,92.4,2018-01-10 19:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390021,A106-0515603020,2018-01-10 16:50:19,2018-03-09 03:01:20,26319659.0,2017-12-28 02:51:36,2018-03-07 13:20:00,False,True,African American or Black,1741485.0,2017-12-21 13:05:34,2017-11-08 11:05:34,True,1.129628e+10,4961461.0,99.000000,97.3,2017-12-03 01:05:34
390022,A106-0515603020,2018-01-10 16:50:19,2018-03-09 03:01:20,26319659.0,2017-12-28 02:51:36,2018-03-07 13:20:00,False,True,African American or Black,1741485.0,2017-12-21 13:05:34,2017-11-08 11:05:34,True,1.129628e+10,4961461.0,99.000000,97.3,2017-12-03 02:05:34
390023,A106-0515603020,2018-01-10 16:50:19,2018-03-09 03:01:20,26319659.0,2017-12-28 02:51:36,2018-03-07 13:20:00,False,True,African American or Black,1741485.0,2017-12-21 13:05:34,2017-11-08 11:05:34,True,1.129628e+10,4961461.0,98.000000,97.3,2017-12-03 03:05:34
390024,A112-0515603023,2018-01-10 16:50:23,2018-03-02 16:11:07,21341087.0,2017-11-30 15:02:07,2018-03-02 08:48:00,True,False,Caucasian or White,-1636.0,2018-01-10 17:17:39,2017-11-25 14:17:39,True,1.042668e+10,4404044.0,NaN,NaN,NaT


In [27]:
(df_merged['DateEnd']=='1969-12-31 23:59:58').mean()

0.030838969709711662

In [28]:
(df_merged['DateEnd']>'1970-12-31 23:59:58').mean()

0.9691610302902883

In [29]:
# df_merged[(df_merged['PAT_ID']==PAT_ID) & (df_merged['deBedTime']=='A140-0515603029')]

# But simply merging them using PAT_ID is not ideal: same PAT_ID may have several deBedTime
## need to filter out rows incorrectly align by checking the timing
## only associate the demographic data to the ox sat. data when the timing match

In [30]:
DateStart = pd.to_datetime(df_merged['DateStart'], format="%Y-%m-%d %H:%M:%S") # str to datetime
DateEnd = pd.to_datetime(df_merged['DateEnd'], format="%Y-%m-%d %H:%M:%S")
times = pd.to_datetime(df_merged['times'], format="%Y-%m-%d %H:%M:%S")

In [31]:
# timedelta: int to datetime
mask = (times-timedelta(minutes=min_threshold) >= DateStart) & (times+timedelta(minutes=min_threshold) <= DateEnd)
df_merged = df_merged[mask]

In [32]:
print(mask.mean())

0.3821847774250947


In [33]:
df_merged['CSN'].unique().shape

(2117,)

In [34]:
df_merged['PAT_ID'].unique().shape

(2011,)

In [35]:
# df_merged

In [36]:
# CSN = df_merged['CSN'].values[0]

In [37]:
# PAT_ID = df_merged['PAT_ID'].unique()[20]
# df = df_merged[df_merged['PAT_ID']==PAT_ID]
# df

In [38]:
# df_oxygen_sat[df_oxygen_sat['PAT_ID']==PAT_ID]

In [39]:
# df_ind[df_ind['PAT_ID']==PAT_ID]

In [40]:
# CSN = df_ind['CSN'].values[0]

In [41]:
# df_oxygen_sat[df_oxygen_sat['csn']==CSN]

In [42]:
# df_oxygen_sat

In [43]:
# df_combined = 

In [44]:
# for PAT_ID in df_oxygen_sat['PAT_ID'].unique():
#     df_pat = df_oxygen_sat[df_oxygen_sat['PAT_ID']==PAT_ID]
#     sys.exit()
    

In [45]:
# df_pat = df_oxygen_sat[df_oxygen_sat['pat_id']==PAT_ID]

In [46]:
# df_pat/

In [47]:
# can't move to dataIO.py
def get_header_valid(inputdir_rec):

    for header_name in sorted(os.listdir(inputdir_rec)):
        if '.hea' not in header_name:
            continue

        if '_' not in header_name:
            continue

        if '_0n' in header_name:
            continue
        if 'layout' in header_name:
            continue

        # print(header_name)
        headerdir = inputdir_rec + header_name


        header_id = header_name.split('_')[-1].split('.hea')[0]


        with open(headerdir, 'r') as f:
            header = f.readlines()

        header_dict = header_extract(header)


        allheader_hours = int(header_id)*8
        currentheader_hours = header_dict['t_dur(hr)']

        # waveform_start = row['BED_LOCATION_START'] + timedelta(hours=allheader_hours)
        # waveform_end = row['BED_LOCATION_START'] + timedelta(hours=allheader_hours) + timedelta(hours=currentheader_hours)

        # using DateStart since it is the start time of the waveform
        waveform_start = row['DateStart'] + timedelta(hours=allheader_hours)
        waveform_end = row['DateStart'] + timedelta(hours=allheader_hours) + timedelta(hours=currentheader_hours)

        header_dict['waveform_start'] = waveform_start
        header_dict['waveform_end'] = waveform_end
        header_dict['header_name'] = header_name
        header_dict['header_id'] = header_id

        if (waveform_start > row['times']) or (waveform_end < row['times']):
            # print('wrong recording')
            continue
        else:
            # print(header_start, header_end, row['times'])
            # print('bingo!')
            return header_dict


        return None
        # print('\t', header_dict['start_time'], header_dict['date'])

    # header_id = header_name.split('_')[-1]

In [48]:
# df_merged.shape

# # df_merged['PAT_ID'].unique().shape

In [49]:
waveform_window = 30
df_demographic_processed = pd.DataFrame()

    
for index, row in df_merged.iterrows():
    
    # 1. get header_dict
    rec_id = row['deBedTime']
    print('\n==================  working on rec_id: {} =================='.format(rec_id) )
    inputdir_rec = inputdir + rec_id + '/'
    header_dict = get_header_valid(inputdir_rec) # None or some dictionary data
    
    # 2. continue if waveform is not within the header_dict timing range
    if header_dict is None:
        print('wrong recording')
        print('\t', row['deBedTime'], row['spo2'], row['sao2'], row['times'], row['Black'], row['White'])
        continue

    # 3. get the waveform data
    df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id'])

    # 4. get only a small piece of the waveform data
    sync_start = row['times'] - timedelta(minutes=waveform_window)
    sync_end = row['times'] + timedelta(minutes=waveform_window)

    i_sync_start = int((sync_start - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])
    i_sync_end = int((sync_end - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])

    df_bed = df_bed[i_sync_start:i_sync_end]
    
    # sys.exit()
    
    # list_time =  [header_dict['waveform_start'] + timedelta(seconds=sec) for sec in df_bed['time']]
    # df_bed['time'] = np.asarray(list_time)
    # mask = (df_bed['time'] >= sync_start) & (df_bed['time'] <= sync_end)
    # df_bed = df_bed[mask]


    # 5. check on the waveform data. Continue if the signals are off

    if 'SPO2' not in list(header_dict['unit_dict'].keys()):
        print('SPO2 channel not in waveform')
        continue
        
    if np.mean(np.diff(df_bed['SPO2'])==0)>signal_quite_threshold:
        print('SPO2 channel is too quiet')
        continue
    if np.mean(np.diff(df_bed['I'])==0)>signal_quite_threshold:
        print('I channel is too quiet')
        continue
    if np.mean(np.diff(df_bed['II'])==0)>signal_quite_threshold:
        print('II channel is too quiet')
        continue
    if np.mean(np.diff(df_bed['III'])==0)>signal_quite_threshold:
        print('III channel is too quiet')
        continue
    if np.mean(np.diff(df_bed['V'])==0)>signal_quite_threshold:
        print('V channel is too quiet')
        continue
        
    # 6. extract features from df_bed
    print('\t\textracting features')
    df_features_bed = get_df_features_bed(df_bed, header_dict, seg_duration=10*60, verbose=False)
    if df_features_bed.shape[0]==0:
        print('\tsignals are too noisy')
        continue
        
    # sys.exit()
    df_features_bed = df_features_bed.drop('i_R_peak', axis=1)
    
    # sys.exit()
    
    # 7. get just the median of the features
    feature_medians = df_features_bed.median()
    feature_medians = pd.DataFrame(feature_medians).T

    
    # 8. combine the demographic and the features!
    row = pd.DataFrame(row).T

    row_processed = pd.concat([row.reset_index(drop=True),feature_medians.reset_index(drop=True)], axis=1 )
    row_processed['index'] = row.index[0]

    print('\tshow df_features_bed size:', df_features_bed.shape)

    df_demographic_processed = pd.concat([df_demographic_processed, row_processed])

    # 9. store the features
    # outputdir_recording = outputdir+rec_id+'/'
    outputdir_recording = outputdir+rec_id+'_'+str(int(row['CSN']))+'/'
    if not os.path.exists(outputdir_recording):
        os.makedirs(outputdir_recording)

    df_features_bed.to_csv(outputdir_recording+'df_features_bed.csv.gz', index=False, compression='gzip')
    
    # sys.exit()
    # 10. store the combined df
    df_demographic_processed.reset_index(drop=True).to_csv(outputdir+'df_demographic_processed.csv.gz', index=False, compression='gzip')

    print('success!')
    
df_demographic_processed.reset_index(drop=True).to_csv(outputdir+'df_demographic_processed.csv.gz', index=False, compression='gzip')




==================  working on rec_id: A140-0515603029 ==================


/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


		extracting features
	signals are too noisy

==================  working on rec_id: A140-0515603029 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================


/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


		extracting features
	signals are too noisy

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================
SPO2 channel is too quiet

==================  working on rec_id: A140-0515620983 ==================


...

		extracting features
	signals are too noisy

==================  working on rec_id: A088-0516752917 ==================
SPO2 channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
		extracting features
	signals are too noisy

==================  working on rec_id: A028-0495319861 ==================
III channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
SPO2 channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
		extracting features
	signals are too noisy

==================  working on rec_id: A028-0495319861 ==================


/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


		extracting features
	signals are too noisy

==================  working on rec_id: A028-0495319861 ==================
V channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
SPO2 channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
V channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
V channel is too quiet

==================  working on rec_id: A028-0495319861 ==================
		extracting features
	signals are too noisy

==================  working on rec_id: A028-0495319861 ==================
		extracting features
	PPG outlier rate: 9.22%
	PPG outlier rate: 6.45%


SystemExit: 

/opt/rh/rh-python38/root/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3348: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# 1/(np.diff(df_features_bed['i_R_peak'].values)/240)*60

In [ ]:
df_features_bed

In [ ]:
plt.plot(df_features_bed['HR'].values)

In [ ]:
# outputdir_recording

In [112]:
# df_features_bed = get_df_features_bed(df_bed, header_dict, seg_duration=10*60, verbose=True)


In [113]:
# df_demographic_processed

In [114]:
# np.mean(np.diff(df_bed['SPO2'])==0), np.mean(np.diff(df_bed['I'])==0), np.mean(np.diff(df_bed['II'])==0), np.mean(np.diff(df_bed['III'])==0), np.mean(np.diff(df_bed['V'])==0)

In [115]:
# df_features_bed = get_df_features_bed(df_bed, header_dict, seg_duration=10*60)


In [116]:
# feature_medians = df_features_bed.median()
# feature_medians = pd.DataFrame(feature_medians).T
# feature_medians

In [117]:
# feature_medians = df_features_bed.median()
# feature_medians = pd.DataFrame(feature_medians).T

# row = pd.DataFrame(row).T

# row_processed = pd.concat([row.reset_index(drop=True),feature_medians.reset_index(drop=True)], axis=1 )
# row_processed['index'] = row.index[0]

In [118]:
# row_processed

In [119]:
# df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id'])

# # 4. get only a small piece of the waveform data
# sync_start = row['times'] - timedelta(minutes=waveform_window)
# sync_end = row['times'] + timedelta(minutes=waveform_window)

# i_sync_start = int((sync_start - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])
# i_sync_end = int((sync_end - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])

# df_bed = df_bed[i_sync_start:i_sync_end]


In [121]:
# list(header_dict['unit_dict'].keys())

In [122]:
# header_dict

In [61]:
sync_start

Timestamp('2017-11-06 20:26:00')

In [60]:
sync_end

Timestamp('2017-11-06 21:26:00')

In [50]:
df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id'])

25946.0

In [67]:
# (sync_end - header_dict['waveform_start']).total_seconds()

29546.0

240

In [79]:
i_sync_start
i_sync_end

7091040

47.56666666666667

In [53]:
# 4. get only a small piece of the waveform data
sync_start = row['times'] - timedelta(minutes=waveform_window)
sync_end = row['times'] + timedelta(minutes=waveform_window)

# list_time =  [header_dict['waveform_start'] + timedelta(seconds=sec) for sec in df_bed['time']]
# df_bed['time'] = np.asarray(list_time)
# mask = (df_bed['time'] >= sync_start) & (df_bed['time'] <= sync_end)
# df_bed = df_bed[mask]

In [54]:
 list_time =  [header_dict['waveform_start'] + timedelta(seconds=sec) for sec in df_bed['time']]


KeyboardInterrupt



In [59]:
dates_list = [datetime(2015,11,i+1) for i in range(df_bed.shape[0])]


ValueError: day is out of range for month

In [58]:
datetime

(6912000, 9)

# TODO: check on df_demographic_processed

In [ ]:
def fix_waveform_datetime(row, header_dict, vervose=False):

    if vervose:
        print('\tdemographic start time: {}'.format(row['DateStart']))
        print('\tdemographic end time: {}'.format(row['DateEnd']))

    date = row['DateStart'].split(' ')[0]
    str_format = '%Y-%m-%d %H:%M:%S.%f'

    waveform_start = datetime.strptime(date + ' ' + header_dict['start_time'], str_format)
    waveform_end = waveform_start + timedelta(hours=header_dict['t_dur(hr)'])

    waveform_start = waveform_start.strftime(str_format)
    waveform_end = waveform_end.strftime(str_format)
    # waveform_start, waveform_end

    header_dict['waveform_start'] = waveform_start
    header_dict['waveform_end'] = waveform_end

    if vervose:
        print('\twaveform start time: {}'.format(header_dict['waveform_start']))
        print('\twaveform end time: {}'.format(header_dict['waveform_end']))

    return header_dict

In [ ]:
df_demographic_processed = pd.DataFrame()

for index, row in df_demographic.iterrows():
    # if index%20!=0:
    #     continue
    if (row['White']==False) & (row['Black']==False):
        continue

    rec_id = row['deBedTime']
    print('working on bed ID: ', rec_id)
    
    # df_bed = get_df_bed(rec_id)
    # sys.exit()
    try:
        df_bed, header_dict = get_df_bed(rec_id, inputdir)
    except:
        continue
    header_dict = fix_waveform_datetime(row, header_dict)


    if 'SPO2' not in list(header_dict['unit_dict'].keys()):
        continue
        
    if np.mean(np.diff(df_bed['SPO2'])==0)>0.3:
        continue
    if np.mean(np.diff(df_bed['I'])==0)>0.3:
        continue
    if np.mean(np.diff(df_bed['II'])==0)>0.3:
        continue
    if np.mean(np.diff(df_bed['III'])==0)>0.3:
        continue
    if np.mean(np.diff(df_bed['V'])==0)>0.3:
        continue

    print('\t\textracting features')
    df_features_bed = get_df_features_bed(df_bed, header_dict, seg_duration=30*60)
    if df_features_bed.shape[0]==0:
        continue
    df_features_bed = df_features_bed.drop('i_R_peak', axis=1)

    feature_medians = df_features_bed.median()
    feature_medians = pd.DataFrame(feature_medians).T

    row = pd.DataFrame(row).T

    row_processed = pd.concat([row.reset_index(drop=True),feature_medians.reset_index(drop=True)], axis=1 )
    row_processed['index'] = row.index[0]

    print('\t\tshow df_features_bed size:', df_features_bed.shape)

    # sys.exit()


    df_demographic_processed = pd.concat([df_demographic_processed, row_processed])
    # df_demographic_processed = pd.concat([df_demographic_processed, row_processed])
    # df_demographic_processed÷
        
        

    # outputdir_recording = outputdir+recording_name+'/'
    outputdir_recording = outputdir+rec_id+'/'
    if not os.path.exists(outputdir_recording):
        os.makedirs(outputdir_recording)

    df_features_bed.to_csv(outputdir_recording+'df_features_bed.csv.gz', index=False, compression='gzip')
    # df = pd.read_csv(outputdir_recording+'df_features_sub.csv.gz', compression='gzip')
    # print(df)
    # sys.exit()
    
df_demographic_processed.to_csv(outputdir+'df_demographic_processed.csv.gz', index=False, compression='gzip')


In [ ]:
# try:
#     df_bed, header_dict = get_df_bed(rec_id, inputdir)
# except:
#     print('jj')

In [ ]:
# inputdir

In [ ]:
# os.listdir(inputdir+'A003-0500955218')

In [ ]:
df_demographic_processed = pd.read_csv(outputdir+'df_demographic_processed.csv.gz')  


In [ ]:
# df_demographic_processed

In [ ]:
df_demographic_processed

id_vars = ['deBedTime', 'DateStart', 'DateEnd', 'PAT_ID', 'CSN', 'BED_LOCATION_START', 'BED_LOCATION_END', 'White', 'Black', 'Race String']

df2 = pd.melt(df_demographic_processed, id_vars=id_vars, 
                  var_name="feature_name", value_name="feature")

In [ ]:
X = df_demographic_processed[ 'B2C', 'max2min', 'PI', 'std', 'skewness', 'kurtosis', 'rms', 'energy', 'mean'].values


In [ ]:
# df_demographic_processed['deBedTime']==

In [ ]:
# ax = sns.boxplot(x="feature_name", y="feature", hue="Race String",
#                  data=tips, linewidth=2.5)

In [ ]:
# row_processed

In [ ]:
# result

In [ ]:
# aaa.index = 800
# aaaa

In [ ]:
# pd.concat([row, aaa], axis=1).reset_index()


In [ ]:
# pd.DataFrame(row).T

In [ ]:
# plot_all_sync(df_seg, rec_id, header_dict, t_start=0, t_end=20, fig_name=None, outputdir=None, show_plot=True)


# signal level functions

In [ ]:
# def get_ecg_IP_norm(ecg1, ecg2):
#     return (ecg1 @ ecg2) / ((ecg1 @ ecg1)**0.5 * (ecg2 @ ecg2)**0.5)



# signal to beats

In [ ]:
# def segment_df(df_seg, QRS_detector_dict, Fs):
    
#     df = get_filt_df(df_seg, Fs).copy()
    
    
#     ecg_dict = QRS_detector_dict['ecg_dict']
#     i_R_peaks = QRS_detector_dict['i_R_peaks']
# #     i_R_peaks = QRS_detector_dict['i_R_peaks']
#     # end_offset = QRS_detector_dict['end_offset']
#     end_offset = int(1*Fs)
#     # print(end_offset)
#     # sys.exit()
    
#     beats_dict = {}
    

#     for sig_name in df.columns:
#         if sig_name =='time':
#             continue
#         if sig_name =='RR':
#             continue
#         sig_beats, i_R_peaks_used = beat_segmentation(df[sig_name].values, i_R_peaks, start_offset=0, end_offset=end_offset)
#         beats_dict[sig_name] = sig_beats
        
        
#     sig_beats, i_R_peaks_used = beat_segmentation(df_seg['SPO2'].values, i_R_peaks, start_offset=0, end_offset=end_offset)
#     beats_dict['PPG_DC'] = sig_beats

#     beats_dict['i_R_peaks'] = i_R_peaks_used
    
#     return beats_dict

# feature level functions

In [ ]:
# def get_B2C(sig,beat_deriv1, beat_deriv2, Fs):
#     N_window = int(0.15*Fs)
#     i_min, i_derivmax, i_foot, data_derivmax, data_min = PPG_argfoot(sig, fs=Fs)
#     # np.argmin(np.abs(beat_deriv1[i_min-N_window:i_min+N_window]))
#     # i_foot = i_max-N_window

#     i_min = i_min[0]

#     i_A = np.argmax(beat_deriv2[i_min:i_min+int(0.1*Fs)])
#     i_A += i_min

#     i_B = np.argmin(beat_deriv2[i_A:i_A+int(0.2*Fs)])
#     i_B += i_A

#     i_C = np.argmax(beat_deriv2[i_B:i_B+int(0.2*Fs)])
#     i_C += i_B

#     i_D = np.argmin(beat_deriv2[i_C:i_C+int(0.2*Fs)])
#     i_D += i_C

#     amplitude_A = np.abs(beat_deriv2[i_A])
#     amplitude_B = np.abs(beat_deriv2[i_B])
#     amplitude_C = np.abs(beat_deriv2[i_C])
#     amplitude_D = np.abs(beat_deriv2[i_D])


#     feature_B2C = amplitude_B/amplitude_C
    
#     output_dict = {
#     'i_min': i_min,
#     'i_A': i_A,
#     'i_B': i_B,
#     'i_C': i_C,
#     'i_D': i_D,
#     'amplitude_A': amplitude_A,
#     'amplitude_B': amplitude_B,
#     'amplitude_C': amplitude_C,
#     'amplitude_D': amplitude_D,
#     'feature_B2C': feature_B2C,
#     }
    
#     return output_dict

# def debug_B2C(beats_PPG, Fs):
#     k = 20
#     sig= beats_PPG[:,[k]]
#     dx = 1/Fs

#     beat_deriv1 = np.gradient(sig.squeeze(), dx)
#     beat_deriv2 = np.gradient(np.gradient(sig.squeeze(), dx), dx)

#     output_dict = get_B2C(sig, beat_deriv1, beat_deriv2, Fs)

#     fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,5), dpi=100)

#     ax1.plot(t_beat, beats[:,k], color='gray', alpha=1)
#     ax1.set_ylabel('PPG')
#     ax2.plot(t_beat, beat_deriv1, color='gray', alpha=1)
#     ax2.set_ylabel('PPG 1st derivative')

#     ax3.plot(t_beat, beat_deriv2, color='gray', alpha=1)

#     ax3.scatter(output_dict['i_min']/Fs, beat_deriv2[output_dict['i_min']])
#     ax3.scatter(output_dict['i_A']/Fs, beat_deriv2[output_dict['i_A']], color='r')
#     ax3.scatter(output_dict['i_B']/Fs, beat_deriv2[output_dict['i_B']], color='g')
#     ax3.scatter(output_dict['i_C']/Fs, beat_deriv2[output_dict['i_C']], color='k')
#     ax3.scatter(output_dict['i_D']/Fs, beat_deriv2[output_dict['i_D']], color='y')
#     ax3.set_ylabel('PPG 2nd derivative')

    
# def get_max2min(sig):

#     feature_max2min = sig.max()-sig.min()

#     output_dict = {
#         'feature_max2min': feature_max2min
#     }

#     return output_dict

# def get_PI(feature_max2min, sig_DC):
    
#     featrue_PI = feature_max2min/sig_DC.mean()
    
#     output_dict = {
#         'featrue_PI': featrue_PI
#     }
    
#     return output_dict


# def get_std(sig):
#     return np.std(sig)

# def get_skewness(sig):
#     return scipy.stats.skew(sig)[0]

# def get_kurtosis(sig):
#     return scipy.stats.kurtosis(sig)[0]

# def get_energy(sig):
#     return np.sum((sig-sig.mean())**2)

# def get_rms(sig):
#     return np.sqrt(np.mean(sig**2))

# def get_mean(sig_DC):
#     return np.mean(sig_DC)


# master beat2features function

In [ ]:
# def beat2features(beats_dict, Fs):

#     beats_PPG = beats_dict['SPO2']
#     beats_PPG_DC = beats_dict['PPG_DC']
#     i_R_peaks = beats_dict['i_R_peaks']
#     dx = 1/Fs

#     df_features = pd.DataFrame()

#     for k in range(beats_PPG.shape[1]):
#         # print(k)
#         sig = beats_PPG[:,[k]]
#         sig_DC = beats_PPG_DC[:,[k]]
#         i_R_peak = i_R_peaks[k]

#         beat_deriv1 = np.gradient(sig.squeeze(), dx)
#         beat_deriv2 = np.gradient(np.gradient(sig.squeeze(), dx), dx)

#         feature_B2C = get_B2C(sig, beat_deriv1, beat_deriv2, Fs)['feature_B2C']
#         feature_max2min = get_max2min(sig)['feature_max2min']
#         featrue_PI = get_PI(feature_max2min, sig_DC)['featrue_PI']



#         # features that use cardiac PPG (filtered between 0.8-4Hz)
#         feature_std = get_std(sig)
#         feature_skewness = get_skewness(sig)
#         feature_kurtosis = get_kurtosis(sig)
#         feature_rms = get_rms(sig)
#         feature_energy = get_energy(sig)


#         # features that use raw PPG
#         feature_mean = get_mean(sig_DC)


#         df_beat = pd.DataFrame({
#             'i_R_peak': [i_R_peak],
#             'B2C': [feature_B2C],
#             'max2min': [feature_max2min],
#             'PI': [featrue_PI],
#             'std': [feature_std],
#             'skewness': [feature_skewness],
#             'kurtosis': [feature_kurtosis],
#             'rms': [feature_rms],
#             'energy': [feature_energy],
#             'mean': [feature_mean],
#         })

#         df_features = pd.concat([df_features, df_beat])

#         # df_features = df_features.append(df_beat)

#     return df_features


In [ ]:
# def get_FQI_dict(data):

#     data_median = medfilt(data, k=9)

#     data_deviation = np.abs(data - data_median)

#     mask_feature = data_deviation < data_deviation.std()*3

#     data_deviation_normed = (data_deviation-data_deviation.min()) / (data_deviation.max()-data_deviation.min())

#     FQI = np.exp(-0.5*data_deviation_normed)

#     FQI_dict = {
#         'data_deviation': data_deviation,
#         'mask_feature': mask_feature,
#         'FQI': FQI,
#     }
#     return FQI_dict

# feature level qaulity assessment

In [ ]:
# def get_mask_all(df_features):

#     mask_feature_list = []
#     for feature_name in df_features.columns:
#         if feature_name == 'i_R_peak':
#             continue

#         FQI_dict = get_FQI_dict(df_features['B2C'].values)
#         mask_feature = FQI_dict['mask_feature']
#         mask_feature_list.append(mask_feature)
#         # print(feature_name)

#     mask_feature_list = np.stack(mask_feature_list).T
#     mask_feature_list = np.prod(mask_feature_list, axis=1)
#     mask_feature_list = mask_feature_list==1
#     return mask_feature_list

In [ ]:
# def masking_beats_dict(beats_dict, mask):

#     # print(beats_dict['AR1'].shape)
#     for beat_name in beats_dict.keys():
#         if beat_name == 'i_R_peaks':
#             beats_dict[beat_name] = beats_dict[beat_name][mask]
#         else:
#             beats_dict[beat_name] = beats_dict[beat_name][:, mask]

#     # print(beats_dict['AR1'].shape)

#     return beats_dict

In [ ]:
# header_dict

In [ ]:
# def get_df_features_bed(df_bed, header_dict, seg_duration=30*60, verbose=False):
#     # df_bed stores the waveform
#     # header_dict stores the meta data
#     # seg_duration: the duration to look at the data (in second)
    
#     # seg_duration = 1 * 60 * 60 # seconds
#     # seg_duration = 30 * 60 # seconds
#     if verbose:
#         print('seg_duration is {} sec'.format(seg_duration))
#     # unit_dict = header_dict['unit_dict']
#     Fs = header_dict['Fs']
#     N_segs = int((df_bed.shape[0]/Fs)//seg_duration)

#     df_features_bed = pd.DataFrame()

#     for i_seg in range(N_segs):

#         if verbose:
#             print('i_seg: ', i_seg)

#         # if i_seg != 5:
#         #     continue
#         i_start = i_seg*seg_duration*Fs
#         i_end = (i_seg+1)*seg_duration*Fs

#         df_seg = df_bed[i_start:i_end]

#         if df_seg.shape[0]==0:
#             continue

#         norm1 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['II']))
#         norm2 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['III']))
#         norm3 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['V']))

#         if (norm1 < 0.3) or (norm2 < 0.3) or (norm3 < 0.3):
#             if verbose:
#                 print('\tbad ECG')
#             continue


#         sig_name = 'I'
#         ECG = df_seg[sig_name].values


#         # ECG = df[sig_name].values

#         fig_name = 'ECG_diagnostics_'+sig_name
#         # QRS_detector_dict = task_HR_detector(ECG, Fs, fig_name=fig_name, outputdir=None, show_plot=True)
#         QRS_detector_dict = task_HR_detector(ECG, Fs, fig_name=fig_name, outputdir=None, show_plot=False)

#     #     t_ecg = np.arange(ECG.shape[0])/Fs
#     #     hr_interp = np.interp(t_ecg, QRS_detector_dict['ts_hr'], QRS_detector_dict['hr'])


#         # segment the beats
#         beats_dict = segment_df(df_seg, QRS_detector_dict, Fs)
#         mask_ppg, ol_rate = clean_PPG(beats_dict['SPO2'], beats_dict['SPO2'].mean(axis=1), Fs)
#         beats_dict = masking_beats_dict(beats_dict, mask_ppg)

#         if ol_rate > 0.1:
#             if verbose:
#                 print('\tbad PPG')
#             continue
#         print('\t\tPPG outlier rate: {:.2f}%'.format( ol_rate*100) )
#         # sys.exit()


#         # sys.exit()

#         if verbose:
#             print('\t\tpass ECG and PPG SQI tests...')


#         beats_dict['i_R_peaks'] = beats_dict['i_R_peaks'] + i_start    

#         # ensemble the beats (N=4)
#         use_ensemble = True

#         if use_ensemble:
#             for key in beats_dict:
#                 if 'ppg' not in key:
#                     continue
#                 sig_beats = beats_dict[key]
#                 beats_dict[key] = get_ensemble_beats(sig_beats, N_enBeats=4, use_woody=False)


#         df_features = beat2features(beats_dict, Fs)

#         # reject unnecessary features
#         mask_all = get_mask_all(df_features)
#         # sys.exit()
#         if verbose:
#             print('\t\tfaeture rejection rate: {:.2f}%'.format( 100-mask_all.mean()*100) )
#         df_features = df_features.loc[mask_all, :].copy()

#         df_features_bed = pd.concat([df_features_bed, df_features])

#     if verbose:
#         print('done')
#         print('show df_features_bed size:', df_features_bed.shape)
    
#     return df_features_bed

In [ ]:
sys.exit()

In [ ]:
beats = beats_dict['SPO2']

plt.plot(beats, color='gray', alpha=0.1)
plt.plot(beats.mean(axis=1), color='red', alpha=0.7)
# plt.ylim(-1,1)
plt.show()

beats = beats_dict['SPO2'][:, mask_all]

plt.plot(beats, color='gray', alpha=0.1)
plt.plot(beats.mean(axis=1), color='red', alpha=0.7)
# plt.ylim(-1,1)
plt.show()

In [ ]:
feature_names = list(df_features_sub.keys())
feature_names.remove('i_R_peak')


# feature_names

In [ ]:
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")

fig, axes = plt.subplots(len(feature_names),1, figsize=(5,20), dpi=80)

for ax, feature_name in zip(axes, feature_names):
# sns.violinplot(x=df_features["B2C"], ax=axes[0], orient='v')
# sns.violinplot(x=df_features["PI"], ax=axes[1], orient='v')

    sns.violinplot(data=df_features_sub, y=feature_name, ax=ax)
# sns.violinplot(data=df_features_sub, y="PI", ax=axes[1])


In [ ]:
df_features_sub.keys()

In [ ]:
plt.plot(df_features_sub['rms'].values)

In [ ]:
np.median(df_features_sub['max2min'].values)

In [ ]:

# np.prod(mask_feature_list, axis=1)

In [ ]:
mask_feature_list.shape

In [ ]:
aaa = np.asarray([[ True,  True,  True, True],
       [ True,  False,  True, True],
       [ True,  True,  True, True]])

aaa

In [ ]:
np.prod(aaa, axis=1)

In [ ]:
mask_feature_list.shape

In [ ]:
# np.stack(mask_feature_list).T
# aaa = np.asarray([[1.,2.],[3.,4.]])
# np.prod(aaa, axis=0), aaa

In [ ]:
FQI_dict

In [ ]:
FQI_dict = get_FQI_dict(df_features['B2C'].values)
plt.plot(FQI_dict['FQI'])

In [ ]:
plt.plot(ECG)
plt.xlim(0,1000)

In [ ]:
t_ecg = np.arange(ECG.shape[0])/Fs
hr_interp = np.interp(t_ecg, QRS_detector_dict['ts_hr'], QRS_detector_dict['hr'])

# plt.plot(t_ecg, hr_interp)
plt.plot(QRS_detector_dict['ts_hr'], QRS_detector_dict['hr'])
plt.xlim(0, 100)

In [ ]:
# plt.plot(t_ecg, ECG)
# plt.xlim(0, 500)

In [ ]:
# plt.plot(t_ecg, ECG)
# plt.xlim(300, 350)

In [ ]:
# plt.plot(t_ecg[::10], hr_interp[::10])
# plt.xlim(0, 1000)

In [ ]:
# def get_filt_df(df_sync, Fs):
#     df = df_sync.copy()

#     for sig_name in df.columns:
#         if 'I' == sig_name or 'II' == sig_name or 'III' == sig_name or 'V' == sig_name:
#             df[sig_name] = get_padded_filt(df[sig_name].values, filter_padded=1, lowcutoff=FILT_ECG[0], highcutoff=FILT_ECG[1], Fs=Fs)
#         elif 'RR' == sig_name:
#             df[sig_name] = get_padded_filt_DSwrapper(df[sig_name].values, filter_padded=1, lowcutoff=FILT_RESP[0], highcutoff=FILT_RESP[1], Fs=Fs)
#         elif 'SPO2' == sig_name:
#             df[sig_name] = get_padded_filt(df[sig_name].values, filter_padded=1, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=Fs)
#         else:
#             pass

#     return df


In [ ]:

# beats_dict = segment_df(df_seg, QRS_detector_dict, Fs)
# use_ensemble = True


In [ ]:
# if use_ensemble:
#     for key in beats_dict:
#         if 'ppg' not in key:
#             continue
#         sig_beats = beats_dict[key]
#         beats_dict[key] = get_ensemble_beats(sig_beats, N_enBeats=4, use_woody=False)

In [ ]:
beats_dict.keys()

In [ ]:
beats_id = [1,2,3,4,5,6]
beats_names = ['I', 'II', 'III', 'V', 'SPO2', 'PPG_DC']
unit_dict['PPG_DC'] = ''

def plot_ALL_beats(beats_dict, beats_id, subject_id, Fs, show_good=None, fig_name=None, outputdir=None, show_plot=False):

    t_beat = np.arange(beats_dict['I'].shape[0])/Fs

    fig = plt.figure(figsize=(16, 10), dpi=80)
    fontsize = 20
    alpha = 0.03


    # for (beat_name, beat_i) in zip(beats_dict, beats_id):
    for (beat_name, beat_i) in zip(beats_names, beats_id):

        beats = beats_dict[beat_name]

        ax = fig.add_subplot(3, 4, beat_i)
        ax.set_title(beat_name+'\n', fontsize=fontsize)

        show_good = True
        if show_good is not None:


            template = beats_dict[beat_name].mean(axis=1)
            if beat_name=='SPO2':
                mask_all, ol_rate = clean_PPG(beats, template, Fs)
                ax.set_title(beat_name+'\nacception_rate:{:.2f}'.format(1-ol_rate), fontsize=fontsize)

            # if 'ppg' in beat_name:

    #             # if beat_name[-1]=='1':
    #                 template = beats_dict['ppg_r_1'].mean(axis=1)
    #             elif beat_name[-1]=='2':
    #                 template = beats_dict['ppg_r_1'].mean(axis=1)

    #             mask_all, ol_rate = clean_PPG(beats, template, Fs)
    #             if show_good==False:
    #                 beats = beats[:, ~mask_all]
    # #                     print(beat_name, ol_rate)
    #                 ax.set_title(beat_name+'\nrejection_rate:{:.2f}'.format(ol_rate), fontsize=fontsize)
    #             else:
    #                 beats = beats[:, mask_all]
    # #                     print(beat_name, ol_rate)
    #                 ax.set_title(beat_name+'\nacception_rate:{:.2f}'.format(1-ol_rate), fontsize=fontsize)



        ax.plot(t_beat, beats, color='gray', alpha=alpha)
        ax.plot(t_beat, np.mean(beats,axis=1), color='firebrick', linewidth=3)

        if 'SPO2' in beat_name or 'I' == beat_name or 'II' == beat_name or 'III' == beat_name or 'V' == beat_name:
            beats_mean = np.mean(beats,axis=1)

            ymin = beats_mean.mean() - beats_mean.std()*5
            ymax = beats_mean.mean() + beats_mean.std()*5
            ax.set_ylim(ymin, ymax)

        ax.tick_params(axis='both', which='major', labelsize=13)
        ax.set_ylabel(unit_dict[beat_name], fontsize=fontsize-3)
        ax.set_xlabel('time (sec)', fontsize=fontsize)



    fig.tight_layout()

    # if outputdir is not None:

    #     if fig_name is None:
    #         fig_name = 'beats_ensemble_sub{}'.format(subject_id)

    #     if not os.path.exists(outputdir):
    #         os.makedirs(outputdir)
    #     fig.savefig(outputdir + fig_name+'.png', transparent=False)

    # if show_plot == False:
    #     plt.close(fig)
    #     pyplot.close(fig)
    #     plt.close('all')


    # #     if log_wandb:
    # #         wandb.log({fig_name: wandb.Image(fig)})



In [ ]:
df_features

In [ ]:
plt.plot(df_features['PI'].values)

In [ ]:
debug_B2C(beats_PPG, Fs)


In [ ]:
data = df_features['B2C'].values
FQI_dict = get_FQI_dict(data)
plt.plot(FQI_dict['FQI'])

In [ ]:
df_features

In [ ]:
data = df_features['PI'].values
FQI_dict = get_FQI_dict(data)
plt.plot(FQI_dict['FQI'])

In [ ]:
data = df_features['max2min'].values
FQI_dict = get_FQI_dict(data)
plt.plot(FQI_dict['FQI'])

In [ ]:
feature_names!='i_R_peak'

In [ ]:
feature_names

In [ ]:
feature_names = df_features.columns
feature_names = feature_names[feature_names!='i_R_peak']
N_feature = feature_names.shape[0]

In [ ]:
df_seg

In [ ]:
fig, (axes) = plt.subplots(N_feature+2, 1, figsize=(20,8), dpi=100)

t_sig = df_seg['time'].values
t_sig = t_sig-t_sig[0]

ax1= axes[0]
ax1.plot(t_sig, df_seg['SPO2'].values)
ax1.set_ylabel('PPG')

ax2= axes[1]
ax2.plot(t_sig, df_seg['RR'].values)
ax2.set_ylabel('RR')

for i, feature_name in enumerate(feature_names):
    ax = axes[i+2]
    # ax2.scatter(beats_dict['i_R_peaks'][mask_feature], features_B2C[mask_feature])
    data = df_features[feature_name].values
    FQI_dict = get_FQI_dict(data)
    mask_feature = FQI_dict['mask_feature']
    ax.plot(df_features['i_R_peak'][mask_feature&mask_all]/Fs, df_features[feature_name][mask_feature&mask_all])
    ax.scatter(df_features['i_R_peak'][mask_feature&mask_all]/Fs, df_features[feature_name][mask_feature&mask_all])
    ax.set_ylabel(feature_name)

ax.set_xlabel('time (s)')
fig.tight_layout()


In [ ]:
beats_dict['i_R_peaks'].shape, features_B2C.shape

In [ ]:
sig_plt

In [ ]:
t_start = 1100
t_end = 1150

df_seg = df[(df['time']>=t_start) & (df['time']<=t_end)].copy()

t_arr = df_seg['time'].values
#     t_arr = t_arr - t_arr[0]

t_start = t_arr[0]
t_end = t_arr[-1]

sig_name = 'SPO2'
sig_plt = df_seg[sig_name].values


sig_plt = get_padded_filt(sig_plt, filter_padded=5, lowcutoff=1, highcutoff=4, Fs=Fs)

# t_arr = df_seg['time'].values
alpha=0.5
fig, ax = plt.subplots(1, 1, figsize=(25,10), gridspec_kw = {'wspace':0, 'hspace':0}, dpi=80)
ax.plot(t_arr, sig_plt, alpha=alpha ,zorder=1, linewidth=linewidth)
# ax.set_ylim(-5,5)

In [ ]:
unit_dict

In [ ]:
homedir = os.path.expanduser('~')+'/'

In [ ]:
inputdir = homedir+'kamaleswaranlab/Sepsis3_062321/Files_cp_sftp_6_30_21/Files_cp_6_30_21/'

In [ ]:
for sub_file in os.listdir(inputdir):
    # print(sub_file)
    pass

In [ ]:
sub_id = 'A071-0512777549'

In [ ]:
for sub_file in os.listdir(inputdir+sub_id):
    if 'hea' in sub_file:
        print(sub_file)